# Add Libraries

In [ ]:
import os
import time
import pandas as pd

from core import core_global as core
from core.spike_monitor import SpikeMonitor
from core.voltage_monitor import VoltageMonitor
from core.weight_matrix_monitor import WeightMatrixMonitor
from layers.rewiring_connection import RewiringConnection
from models.poisson_pattern_group import PoissonPatternGroup
from utils import utils as utils
from models.mc_lif_group import McLifGroup

# Run Exploration Phase

In [ ]:
def main(args):
    trial = args[0]
    config = args[1]
    input_params = config["input_parameters"]
    connection_params = config["connection_parameters"]
    neuron_params = config["neuron_parameters"]
    
    
    
    print("##############Parameters#############")
    print("Network_size: (CA3, CA1) = (", input_params['num_inputs'],",",config["num_neurons"],")")
    print("Simulation_time:",config['simulation_time']," s")
    print("####################################")
    

    exp_result_name_folder = "Exploration_Phase"

    
    
    ##############################################################

    # Directory for simulation results and log files.
    output_directory = os.path.join("results", "rewiring_ex1",exp_result_name_folder, time.strftime("%y%m%d_%H%M%S"), str(trial),
                                    "data")

    # Initialize the simulation environment.
    core.init(directory=output_directory)

    # Write config file to the output directory.
    utils.write_configuration(os.path.join(output_directory, "..", "initialization.yaml"), config)

    # Set the random seed.
    core.kernel.set_master_seed(config["master_seed"])

    # Create input neurons.
    inp = PoissonPatternGroup(input_params["num_inputs"], input_params["rate"], input_params["rate_bg"],
                              params=input_params)

    # Create the network.
    neuron = McLifGroup(config['num_neurons'], neuron_params["num_branches"], neuron_params)

    # Creat the Connections.
    conn = RewiringConnection(inp, neuron, neuron.branch.syn_current, connection_params)

    # Create some monitors which will record the simulation data.
    for cell_num in range(config['num_neurons']):
        WeightMatrixMonitor(conn, core.kernel.fn(f"cell_recording/cell_num_{cell_num}/weights", "dat"),cell_id = cell_num ,
                    interval=config["sampling_interval_weights"])

        
    
    SpikeMonitor(neuron, core.kernel.fn("output", "ras"))
    sm_inp = SpikeMonitor(inp, core.kernel.fn("input", "ras"))
    vm_nrn = [VoltageMonitor(neuron.soma, cell_ind, core.kernel.fn(f"cell_recording/cell_num_{cell_ind}/soma", "mem"),paste_spikes=True, paste_spikes_from=neuron) for cell_ind in range(config['num_neurons'])]
    vm_br = [[] for cell_ind in range(config['num_neurons'])]
    for cell_ind in range(config['num_neurons']):
        for dend_ind in range(neuron_params["num_branches"]):
            vm_br[cell_ind].append(VoltageMonitor(neuron.branch, (dend_ind, cell_ind), core.kernel.fn(f"cell_recording/cell_num_{cell_ind}/branch", "mem", dend_ind)))

    # Now simulate the model.
    simulation_time = config["simulation_time"]
    core.kernel.run_chunk(20.0, 0, simulation_time)

    sm_inp.active = True
    for cell_ind in range(config['num_neurons']):
        vm_nrn[cell_ind].active = False
        for vm in vm_br[cell_ind]:
            vm.active = False
            
    core.kernel.run_chunk(simulation_time - 120, 0, simulation_time)

    sm_inp.active = True
    for cell_ind in range(config['num_neurons']):
        vm_nrn[cell_ind].active = True
        for vm in vm_br[cell_ind]:
            vm.active = True

    core.kernel.run_chunk(100.0, 0, simulation_time)


In [ ]:
if __name__ == '__main__':
    import copy
    from scoop import futures

    # Load the configuration file.
    config = utils.load_configuration("initialization.yaml")

    configs = []
    num_trials = 1
    for trial in range(num_trials):
        config["master_seed"] = 10 * (trial + 1)
        configs.append(copy.deepcopy(config))


    r = list(futures.map(main, [[trial, config] for trial, config in enumerate(configs)]))